In [2]:
# Install TensorFlow with GPU support and plotting libraries
%pip install "tensorflow[and-cuda]" numpy matplotlib scipy

  Using cached scipy-1.17.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached tensorflow-2.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
  Using cached absl_py-2.4.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.12.19-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached gast-0.7.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-6.33.5-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached setuptools-80.10.2-py3-none-any.whl.metadata (6.6 kB)
  Using cached termcolor-3.3.0-py3-none-any.whl.metadata (6.5 kB)


In [3]:
import sys

# 1. Print where the notebook is actually running from
print(f"Running from: {sys.executable}")

# 2. Force install into THIS specific python
!{sys.executable} -m pip install "tensorflow[and-cuda]" numpy matplotlib scipy

Running from: /home/taksh/Documents/code/research/venv/bin/python


## 1. Imports & Configuration

In [ ]:
import os
import tarfile
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Tuple, Dict, Callable

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import VGG19, ResNet101, DenseNet121
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

2026-02-02 11:46:09.654722: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-02 11:46:09.705642: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-02 11:46:10.663148: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


TensorFlow version: 2.20.0
Keras version: 3.13.2


## 2. GPU Configuration & Mixed Precision

In [5]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Found {len(gpus)} GPU(s): {[gpu.name for gpu in gpus]}")
        print("Memory growth enabled for all GPUs")
    except RuntimeError as e:
        print(f"GPU configuration error: {e}")
else:
    print("WARNING: No GPU found. Training will be slow on CPU.")

tf.keras.mixed_precision.set_global_policy('mixed_float16')
print(f"Mixed precision policy: {tf.keras.mixed_precision.global_policy().name}")

Found 1 GPU(s): ['/physical_device:GPU:0']
Memory growth enabled for all GPUs
Mixed precision policy: mixed_float16


## 3. Global Configuration

In [ ]:
CONFIG = {
    'zip_file': 'caltech-101.zip',  # Changed from tar_file
    'extract_dir': 'caltech101_data',
    'img_size': (224, 224),
    'batch_size': 32,
    'epochs': 50,
    'train_split': 0.8,
    'val_split': 0.1,
    'test_split': 0.1,
    'seed': 42,
    'learning_rate': 1e-4,
    'patience_early_stop': 10,
    'patience_lr_reduce': 5,
}

MODELS_TO_TRAIN = ['VGG19', 'ResNet101', 'DenseNet121']
CHECKPOINT_DIR = Path('checkpoints')
CHECKPOINT_DIR.mkdir(exist_ok=True)

np.random.seed(CONFIG['seed'])
tf.random.set_seed(CONFIG['seed'])

In [ ]:
def extract_zip_if_needed(zip_path: str, extract_dir: str) -> Path:
    """Extract zip file if not already extracted."""
    extract_path = Path(extract_dir)
    
    if extract_path.exists() and any(extract_path.iterdir()):
        print(f"Data already extracted at: {extract_path}")
        return extract_path
    
    if not Path(zip_path).exists():
        raise FileNotFoundError(f"Zip file not found: {zip_path}")
    
    print(f"Extracting {zip_path}...")
    extract_path.mkdir(parents=True, exist_ok=True)
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(path=extract_path)
    
    print(f"Extraction complete: {extract_path}")
    return extract_path

data_root = extract_zip_if_needed(CONFIG['zip_file'], CONFIG['extract_dir'])

def find_image_root(base_path: Path) -> Path:
    """Find the actual root containing class folders with images."""
    # First, look for common Caltech-101 directory structures
    possible_roots = [
        base_path / '101_ObjectCategories',
        base_path / 'caltech-101' / '101_ObjectCategories',
        base_path / 'caltech101' / '101_ObjectCategories',
    ]
    
    for root in possible_roots:
        if root.exists() and root.is_dir():
            return root
    
    # Fallback: search for image files
    for item in base_path.rglob('*'):
        if item.is_file() and item.suffix.lower() in ['.jpg', '.jpeg', '.png', '.gif', '.bmp']:
            return item.parent.parent
    
    # Another fallback: find directory with subdirectories
    for subdir in base_path.iterdir():
        if subdir.is_dir():
            subdirs = list(subdir.iterdir())
            if subdirs and all(s.is_dir() for s in subdirs[:5]):
                return subdir
    return base_path

IMAGE_ROOT = find_image_root(data_root)
print(f"Image root directory: {IMAGE_ROOT}")

# List contents to verify
print(f"\nContents of extract directory:")
for item in sorted(data_root.iterdir())[:10]:
    print(f"  {item.name}")

FileNotFoundError: Tar file not found: Caltech_WebFaces.tar

## 5. Dataset Discovery & Statistics

In [ ]:
def get_class_names_and_counts(image_root: Path) -> Tuple[list, dict]:
    """Get class names and image counts per class."""
    class_names = sorted([d.name for d in image_root.iterdir() if d.is_dir()])
    class_counts = {}
    
    for class_name in class_names:
        class_dir = image_root / class_name
        count = len([f for f in class_dir.iterdir() 
                     if f.is_file() and f.suffix.lower() in ['.jpg', '.jpeg', '.png', '.gif', '.bmp']])
        class_counts[class_name] = count
    
    return class_names, class_counts

CLASS_NAMES, CLASS_COUNTS = get_class_names_and_counts(IMAGE_ROOT)
NUM_CLASSES = len(CLASS_NAMES)
TOTAL_IMAGES = sum(CLASS_COUNTS.values())

print(f"Number of classes: {NUM_CLASSES}")
print(f"Total images: {TOTAL_IMAGES}")
print(f"\nSample classes: {CLASS_NAMES[:10]}...")
print(f"\nClass distribution (first 10):")
for cls in list(CLASS_COUNTS.keys())[:10]:
    print(f"  {cls}: {CLASS_COUNTS[cls]} images")

## 6. Create tf.data.Dataset with Train/Val/Test Splits

In [ ]:
def collect_all_image_paths_and_labels(image_root: Path, class_names: list) -> Tuple[np.ndarray, np.ndarray]:
    """Collect all image paths and their corresponding labels."""
    all_paths = []
    all_labels = []
    class_to_idx = {name: idx for idx, name in enumerate(class_names)}
    
    for class_name in class_names:
        class_dir = image_root / class_name
        for img_path in class_dir.iterdir():
            if img_path.is_file() and img_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.gif', '.bmp']:
                all_paths.append(str(img_path))
                all_labels.append(class_to_idx[class_name])
    
    return np.array(all_paths), np.array(all_labels)

all_paths, all_labels = collect_all_image_paths_and_labels(IMAGE_ROOT, CLASS_NAMES)

indices = np.arange(len(all_paths))
np.random.shuffle(indices)
all_paths = all_paths[indices]
all_labels = all_labels[indices]

train_size = int(CONFIG['train_split'] * len(all_paths))
val_size = int(CONFIG['val_split'] * len(all_paths))

train_paths, train_labels = all_paths[:train_size], all_labels[:train_size]
val_paths, val_labels = all_paths[train_size:train_size+val_size], all_labels[train_size:train_size+val_size]
test_paths, test_labels = all_paths[train_size+val_size:], all_labels[train_size+val_size:]

print(f"Train samples: {len(train_paths)}")
print(f"Validation samples: {len(val_paths)}")
print(f"Test samples: {len(test_paths)}")

## 7. Dataset Pipeline Factory (with Model-Specific Preprocessing)

In [ ]:
PREPROCESS_FUNCTIONS = {
    'VGG19': vgg_preprocess,
    'ResNet101': resnet_preprocess,
    'DenseNet121': densenet_preprocess,
}

def create_dataset_pipeline(
    paths: np.ndarray,
    labels: np.ndarray,
    preprocess_fn: Callable,
    img_size: Tuple[int, int],
    batch_size: int,
    shuffle: bool = False,
    augment: bool = False
) -> tf.data.Dataset:
    """Create an optimized tf.data pipeline with model-specific preprocessing."""
    
    def load_and_preprocess(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, img_size)
        img = tf.cast(img, tf.float32)
        img = preprocess_fn(img)
        return img, label
    
    def augment_image(img, label):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_brightness(img, 0.1)
        img = tf.image.random_contrast(img, 0.9, 1.1)
        return img, label
    
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(paths), seed=CONFIG['seed'])
    
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    
    if augment:
        dataset = dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset

def get_datasets_for_model(model_name: str) -> Tuple[tf.data.Dataset, tf.data.Dataset, tf.data.Dataset]:
    """Create train, val, test datasets with model-specific preprocessing."""
    preprocess_fn = PREPROCESS_FUNCTIONS[model_name]
    
    train_ds = create_dataset_pipeline(
        train_paths, train_labels, preprocess_fn,
        CONFIG['img_size'], CONFIG['batch_size'],
        shuffle=True, augment=True
    )
    val_ds = create_dataset_pipeline(
        val_paths, val_labels, preprocess_fn,
        CONFIG['img_size'], CONFIG['batch_size'],
        shuffle=False, augment=False
    )
    test_ds = create_dataset_pipeline(
        test_paths, test_labels, preprocess_fn,
        CONFIG['img_size'], CONFIG['batch_size'],
        shuffle=False, augment=False
    )
    
    return train_ds, val_ds, test_ds

print("Dataset pipeline factory created.")

## 8. Model Factory

In [ ]:
BASE_MODELS = {
    'VGG19': VGG19,
    'ResNet101': ResNet101,
    'DenseNet121': DenseNet121,
}

def create_model(model_name: str, num_classes: int, img_size: Tuple[int, int]) -> Model:
    """Create a transfer learning model with trainable classification head."""
    
    base_model_class = BASE_MODELS[model_name]
    
    base_model = base_model_class(
        weights='imagenet',
        include_top=False,
        input_shape=(*img_size, 3),
        pooling=None
    )
    
    base_model.trainable = False
    
    inputs = keras.Input(shape=(*img_size, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
    
    model = Model(inputs, outputs, name=f"{model_name}_transfer")
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=CONFIG['learning_rate']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

print("Model factory created.")
print(f"Models available: {list(BASE_MODELS.keys())}")

## 9. Callbacks Factory

In [ ]:
def get_callbacks(model_name: str) -> list:
    """Create callbacks for training."""
    
    checkpoint_path = CHECKPOINT_DIR / f"{model_name}_best.keras"
    
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=CONFIG['patience_early_stop'],
            restore_best_weights=True,
            verbose=1
        ),
        ModelCheckpoint(
            filepath=str(checkpoint_path),
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=CONFIG['patience_lr_reduce'],
            min_lr=1e-7,
            verbose=1
        ),
    ]
    
    return callbacks

print("Callbacks factory created.")

## 10. Training Loop

In [ ]:
training_histories: Dict[str, keras.callbacks.History] = {}
trained_models: Dict[str, Model] = {}

for model_name in MODELS_TO_TRAIN:
    print("\n" + "="*80)
    print(f"Training {model_name}")
    print("="*80)
    
    tf.keras.backend.clear_session()
    
    train_ds, val_ds, test_ds = get_datasets_for_model(model_name)
    
    model = create_model(model_name, NUM_CLASSES, CONFIG['img_size'])
    
    print(f"\n{model_name} Summary:")
    print(f"  Total params: {model.count_params():,}")
    trainable_params = sum([tf.reduce_prod(v.shape).numpy() for v in model.trainable_variables])
    print(f"  Trainable params: {trainable_params:,}")
    
    callbacks = get_callbacks(model_name)
    
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=CONFIG['epochs'],
        callbacks=callbacks,
        verbose=1
    )
    
    training_histories[model_name] = history
    trained_models[model_name] = model
    
    print(f"\n{model_name} training complete!")

print("\n" + "="*80)
print("ALL MODELS TRAINED SUCCESSFULLY")
print("="*80)

## 11. Evaluation on Test Set

In [ ]:
print("\n" + "="*80)
print("TEST SET EVALUATION")
print("="*80)

test_results = {}

for model_name in MODELS_TO_TRAIN:
    print(f"\nEvaluating {model_name}...")
    
    _, _, test_ds = get_datasets_for_model(model_name)
    
    model = trained_models[model_name]
    
    loss, accuracy = model.evaluate(test_ds, verbose=0)
    
    test_results[model_name] = {
        'loss': loss,
        'accuracy': accuracy
    }
    
    print(f"  Test Loss: {loss:.4f}")
    print(f"  Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

print("\n" + "="*80)
print("FINAL RESULTS SUMMARY")
print("="*80)
print(f"\n{'Model':<15} {'Test Loss':<12} {'Test Accuracy':<15}")
print("-" * 42)
for model_name, results in test_results.items():
    print(f"{model_name:<15} {results['loss']:<12.4f} {results['accuracy']*100:.2f}%")

best_model = max(test_results.keys(), key=lambda k: test_results[k]['accuracy'])
print(f"\nBest performing model: {best_model} with {test_results[best_model]['accuracy']*100:.2f}% accuracy")

## 12. Training Curves Visualization

In [ ]:
def plot_training_history(histories: Dict[str, keras.callbacks.History], metric: str = 'accuracy'):
    """Plot training curves for all models."""
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = range(1, len(history.history[metric]) + 1)
        
        axes[0].plot(epochs, history.history[metric], 
                     color=colors[idx], linestyle='-', linewidth=2,
                     label=f'{model_name} (Train)')
        axes[0].plot(epochs, history.history[f'val_{metric}'], 
                     color=colors[idx], linestyle='--', linewidth=2,
                     label=f'{model_name} (Val)')
    
    axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Accuracy', fontsize=12)
    axes[0].legend(loc='lower right', fontsize=10)
    axes[0].grid(True, alpha=0.3)
    axes[0].set_ylim([0, 1])
    
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = range(1, len(history.history['loss']) + 1)
        
        axes[1].plot(epochs, history.history['loss'], 
                     color=colors[idx], linestyle='-', linewidth=2,
                     label=f'{model_name} (Train)')
        axes[1].plot(epochs, history.history['val_loss'], 
                     color=colors[idx], linestyle='--', linewidth=2,
                     label=f'{model_name} (Val)')
    
    axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].legend(loc='upper right', fontsize=10)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Training curves saved to: training_curves.png")

plot_training_history(training_histories)

## 13. Individual Model Training Curves

In [ ]:
def plot_individual_model_curves(histories: Dict[str, keras.callbacks.History]):
    """Plot individual training curves for each model."""
    n_models = len(histories)
    fig, axes = plt.subplots(n_models, 2, figsize=(14, 5 * n_models))
    
    if n_models == 1:
        axes = axes.reshape(1, -1)
    
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = range(1, len(history.history['accuracy']) + 1)
        
        axes[idx, 0].plot(epochs, history.history['accuracy'], 'b-', linewidth=2, label='Train')
        axes[idx, 0].plot(epochs, history.history['val_accuracy'], 'r--', linewidth=2, label='Validation')
        axes[idx, 0].set_title(f'{model_name} - Accuracy', fontsize=12, fontweight='bold')
        axes[idx, 0].set_xlabel('Epoch')
        axes[idx, 0].set_ylabel('Accuracy')
        axes[idx, 0].legend()
        axes[idx, 0].grid(True, alpha=0.3)
        axes[idx, 0].set_ylim([0, 1])
        
        axes[idx, 1].plot(epochs, history.history['loss'], 'b-', linewidth=2, label='Train')
        axes[idx, 1].plot(epochs, history.history['val_loss'], 'r--', linewidth=2, label='Validation')
        axes[idx, 1].set_title(f'{model_name} - Loss', fontsize=12, fontweight='bold')
        axes[idx, 1].set_xlabel('Epoch')
        axes[idx, 1].set_ylabel('Loss')
        axes[idx, 1].legend()
        axes[idx, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('individual_training_curves.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Individual curves saved to: individual_training_curves.png")

plot_individual_model_curves(training_histories)

## 14. Model Comparison Bar Chart

In [ ]:
def plot_model_comparison(test_results: dict):
    """Bar chart comparing test accuracy across models."""
    models = list(test_results.keys())
    accuracies = [test_results[m]['accuracy'] * 100 for m in models]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = ['#3498db', '#e74c3c', '#2ecc71']
    bars = ax.bar(models, accuracies, color=colors, edgecolor='black', linewidth=1.2)
    
    for bar, acc in zip(bars, accuracies):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{acc:.2f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    ax.set_ylabel('Test Accuracy (%)', fontsize=12)
    ax.set_xlabel('Model', fontsize=12)
    ax.set_title('Model Comparison - Test Accuracy', fontsize=14, fontweight='bold')
    ax.set_ylim([0, 100])
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Model comparison saved to: model_comparison.png")

plot_model_comparison(test_results)

## 15. Save Final Models

In [ ]:
FINAL_MODELS_DIR = Path('final_models')
FINAL_MODELS_DIR.mkdir(exist_ok=True)

for model_name, model in trained_models.items():
    save_path = FINAL_MODELS_DIR / f"{model_name}_final.keras"
    model.save(save_path)
    print(f"Saved {model_name} to: {save_path}")

class_names_path = FINAL_MODELS_DIR / 'class_names.txt'
with open(class_names_path, 'w') as f:
    for name in CLASS_NAMES:
        f.write(f"{name}\n")
print(f"Saved class names to: {class_names_path}")

print("\n" + "="*80)
print("TRAINING PIPELINE COMPLETE")
print("="*80)
print(f"\nCheckpoints directory: {CHECKPOINT_DIR}")
print(f"Final models directory: {FINAL_MODELS_DIR}")
print(f"Visualization files: training_curves.png, individual_training_curves.png, model_comparison.png")